In [692]:
%load_ext autoreload
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [693]:
%pip install -e ../ --quiet

Note: you may need to restart the kernel to use updated packages.


Log into Docker registry with ECR credentials

In [694]:
!aws ecr get-login-password --region eu-west-1 | docker login --username AWS --password-stdin 141502667606.dkr.ecr.eu-west-1.amazonaws.com

Login Succeeded


In [695]:
!mkdir -p pipelines/recommendations/code

In [696]:
import os
import sys

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [697]:
%%writefile pipelines/recommendations/code/user_item_matrix_transformer.py

import numpy as np
import pandas as pd
import logging
import os
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics.pairwise import cosine_similarity

logging.basicConfig(level=logging.INFO)


class UserItemMatrixTransformer(TransformerMixin, BaseEstimator):
    """
    This class is a custom scikit-learn transformer
    that accepts a pandas dataframe of user/item interactions
    and returns a user/item matrix.

    :param user (str): Column name for user id
    :param item (str): Column name for item id
    :param rating (float): Column name for user/item rating
    :param agg (str): Panadas aggregation function to use when combining duplicate user/item interactions
    :param binary (bool): If True, user/item interactions are converted to binary values in the user/item output matrix
    """

    def __init__(
        self, user="user_id", item="item_id", rating="rating", agg="max", binary=False
    ):
        self.user = user
        self.item = item
        self.rating = rating
        self.agg = agg
        self.binary = binary

    def fit(self, X, y=None):
        return self

    def transform(self, X: pd.DataFrame):
        matrix = X.groupby([self.user, self.item])[self.rating].agg(self.agg).unstack()
        if self.binary:
            return matrix.notnull().astype(int)
        else:
            return matrix.fillna(0)


if __name__ == "__main__":
    base_dir = "/opt/ml/processing"
    data_types = {"user_id": str, "item_id": str, "rating": np.float64}

    logging.info(os.listdir(base_dir))
    logging.info(os.listdir(f"{base_dir}/input"))

    user_item_ratings = pd.read_csv(f"{base_dir}/input/data/user_item_ratings.csv", dtype=data_types, engine='python')
    
    transformer = UserItemMatrixTransformer()
    user_item_matrix = transformer.transform(user_item_ratings)

    user_item_matrix.to_csv(f"{base_dir}/output/data/user_item_matrix.csv", header=True, index=False)

Overwriting pipelines/recommendations/code/user_item_matrix_transformer.py


In [698]:
%%writefile pipelines/recommendations/code/similarity_matrix_transformer.py

import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics.pairwise import cosine_similarity
import argparse

class SimilarityTransformer(TransformerMixin, BaseEstimator):
    """
    This class is a custom scikit-learn transformer
    that accepts a user/item matrix where user ids are
    the index and item ids are the columns and returns
    a similarity matrix. It can be used to calculate
    user-user or item-item similarity.
    """

    def __init__(self, kind="user", metric="cosine", normalise=False):
        if kind not in ["user", "item"]:
            raise ValueError("kind must be 'user' or 'item'")
        if metric not in ["cosine", "dot", "euclidean"]:
            raise ValueError("metric must be 'cosine', 'dot', or 'euclidean'")
        self.kind = kind
        self.metric = metric
        self.normalise = normalise

    def fit(self, X, y=None):
        return self

    def transform(self, X: pd.DataFrame):
        matrix = X
        if self.kind == "item":
            matrix = X.T

        if self.metric == "cosine":
            df = pd.DataFrame(
                cosine_similarity(matrix), index=matrix.index, columns=matrix.index
            )
        else:
            raise NotImplementedError("Only cosine similarity is currently supported")

        if self.normalise:
            df = (df - df.min()) / (df.max() - df.min())

        return df



parser = argparse.ArgumentParser()
parser.add_argument("--kind", type=str, default="user")
parser.add_argument("--metric", type=str, default="cosine")
args = parser.parse_args()

if __name__ == "__main__":
    base_dir = "/opt/ml/processing"

    user_item_matrix = pd.read_csv(f"{base_dir}/input/data/user_item_matrix.csv", dtype=np.float64)
    
    transformer = SimilarityTransformer(kind=args.kind, metric=args.metric)
    similarity_matrix = transformer.transform(user_item_matrix)

    similarity_matrix.to_csv(f"{base_dir}/output/data/{args.kind}_similarity_matrix.csv", header=True, index=False)

Overwriting pipelines/recommendations/code/similarity_matrix_transformer.py


In [699]:
%%writefile pipelines/recommendations/code/item_recommender_train.py



Overwriting pipelines/recommendations/code/item_recommender_train.py


In [700]:
import sagemaker
import boto3

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client("iam")
    role = iam.get_role(RoleName="pipeliner")["Role"]["Arn"]

role

'arn:aws:iam::477807511636:role/pipeliner'

In [701]:
session = LocalPipelineSession()
session.config = {"local": {"local_code": True}}

region = session.boto_region_name
default_bucket = session.default_bucket()

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


In [702]:
import pandas as pd
import numpy as np

ratings_data_path = "../tests/test_data/user_item_ratings.csv"
data_types = {"user_id": str, "item_id": str, "rating": np.float64}

user_item_ratings = pd.read_csv(ratings_data_path, dtype=data_types, engine='python')
user_item_ratings.head(5)

,user_id,item_id,rating
0,U1000,I1063,0.36
1,U1001,I1062,0.79
2,U1001,I1099,0.43
3,U1002,I1043,0.64
4,U1002,I1013,0.43


In [703]:
input_data_uri = sagemaker.s3.S3Uploader.upload(
    local_path=ratings_data_path,
    desired_s3_uri=f"s3://{default_bucket}/recommendations/data",
)
input_data_uri

's3://sagemaker-eu-west-1-477807511636/recommendations/data/user_item_ratings.csv'

In [704]:
from sagemaker.workflow.pipeline_context import LocalPipelineSession
import sagemaker
from sagemaker import ScriptProcessor
from sagemaker.workflow.pipeline_context import LocalPipelineSession
from sagemaker.workflow.parameters import ParameterString
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.sklearn.estimator import SKLearn
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import CacheConfig, ProcessingStep, TrainingStep


class RecommenderPipeline:
    def create(
        self,
        role: str,
        name: str,
        session: sagemaker.Session,
        framework_version = "1.2-1",
    ) -> Pipeline:
        self.local = isinstance(session, LocalPipelineSession)
        self.framework_version = framework_version

        instance_type = ParameterString(
            name="InstanceType",
            default_value="local" if self.local else "ml.m5.large",
        )
        
        input_data = ParameterString(
            name="user_item_ratings",
            default_value=input_data_uri,
        )

        image_uri = sagemaker.image_uris.retrieve(
            framework="sklearn",
            region=session.boto_region_name,
            version="1.2-1",
        )

        cache_config = CacheConfig(
            enable_caching=True,
            expire_after="P30d",  # 30 days
        )

        processor = SKLearnProcessor(
            framework_version=framework_version,
            instance_type=instance_type,
            instance_count=1,
            base_job_name="sklearn-preprocess",
            role=role,
            sagemaker_session=session,
        )

        user_item_matrix_step = ProcessingStep(
            name="user_item_matrix_transformer",
            step_args=processor.run(
                inputs=[
                    ProcessingInput(
                        source=input_data, 
                        input_name="user_item_ratings", 
                        destination="/opt/ml/processing/input/data"),
                ],
                outputs=[
                    ProcessingOutput(
                        output_name="user_item_matrix", 
                        source="/opt/ml/processing/output/data"),
                ],
                code="pipelines/recommendations/code/user_item_matrix_transformer.py",
            ),
        )

        user_similarity_matrix_step = ProcessingStep(
            name="user_similarity_matrix_transformer",
            step_args=processor.run(
                inputs=[
                    ProcessingInput(
                        source=user_item_matrix_step.properties.ProcessingOutputConfig.Outputs["user_item_matrix"].S3Output.S3Uri,
                        input_name="user_item_matrix",
                        destination="/opt/ml/processing/input/data"),
                ],
                outputs=[
                    ProcessingOutput(
                        output_name="user_similarity_matrix", 
                        source="/opt/ml/processing/output/data"),
                ],
                code="pipelines/recommendations/code/similarity_matrix_transformer.py",
            ),
            job_arguments=["--kind", "user"],
        )

        item_similarity_matrix_step = ProcessingStep(
            name="item_similarity_matrix_transformer",
            step_args=processor.run(
                inputs=[
                    ProcessingInput(
                        source=user_item_matrix_step.properties.ProcessingOutputConfig.Outputs["user_item_matrix"].S3Output.S3Uri,
                        input_name="user_item_matrix",
                        destination="/opt/ml/processing/input/data"),
                ],
                outputs=[
                    ProcessingOutput(
                        output_name="item_similarity_matrix", 
                        source="/opt/ml/processing/output/data"),
                ],
                code="pipelines/recommendations/code/similarity_matrix_transformer.py",
            ),
            job_arguments=["--kind", "item"],
        )

        # sklearn_estimator = SKLearn(
        #     entry_point="pipelines/recommendations/code/item_recommender_train.py",
        #     role=role,
        #     image_uri=image_uri,
        #     instance_type=instance_type,
        #     sagemaker_session=session,
        #     base_job_name="training_job",
        #     # hyperparameters=hyperparameters,
        #     enable_sagemaker_metrics=True,
        # )

        # training_step = TrainingStep(
        #     name="Train", estimator=sklearn_estimator, cache_config=cache_config
        # )

        return Pipeline(
            name=name,
            # steps=[user_item_matrix_step, item_similarity_matrix_step, training_step],
            steps=[
                user_item_matrix_step, 
                user_similarity_matrix_step, 
                item_similarity_matrix_step
            ],
            sagemaker_session=session,
            parameters=[input_data, instance_type],
        )

In [705]:
pipeline = RecommenderPipeline().create(role=role, name="recommender", session=session)

INFO:sagemaker.image_uris:Defaulting to only available Python version: py3
INFO:sagemaker.image_uris:Defaulting to only supported image scope: cpu.
INFO:sagemaker.image_uris:Defaulting to only available Python version: py3
/Users/robhill/miniconda3/envs/pipeliner/lib/python3.12/site-packages/sagemaker/workflow/pipeline_context.py:332: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(


In [706]:
import json

definition = json.loads(pipeline.definition())
definition

{'Version': '2020-12-01',
 'Metadata': {},
 'Parameters': [{'Name': 'user_item_ratings',
   'Type': 'String',
   'DefaultValue': 's3://sagemaker-eu-west-1-477807511636/recommendations/data/user_item_ratings.csv'},
  {'Name': 'InstanceType', 'Type': 'String', 'DefaultValue': 'local'}],
 'PipelineExperimentConfig': {'ExperimentName': {'Get': 'Execution.PipelineName'},
  'TrialName': {'Get': 'Execution.PipelineExecutionId'}},
 'Steps': [{'Name': 'user_item_matrix_transformer',
   'Type': 'Processing',
   'Arguments': {'ProcessingResources': {'ClusterConfig': {'InstanceType': {'Get': 'Parameters.InstanceType'},
      'InstanceCount': 1,
      'VolumeSizeInGB': 30}},
    'AppSpecification': {'ImageUri': '141502667606.dkr.ecr.eu-west-1.amazonaws.com/sagemaker-scikit-learn:1.2-1-cpu-py3',
     'ContainerEntrypoint': ['python3',
      '/opt/ml/processing/input/code/user_item_matrix_transformer.py']},
    'RoleArn': 'arn:aws:iam::477807511636:role/pipeliner',
    'ProcessingInputs': [{'InputNam

In [707]:
pipeline.upsert(role_arn=role)

INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.


{'PipelineArn': 'recommender'}

In [708]:
execution = pipeline.start()

INFO:sagemaker.local.entities:Starting execution for pipeline recommender. Execution ID is 55497159-63ff-4182-a491-c3a80d9387db
INFO:sagemaker.local.entities:Starting pipeline step: 'user_item_matrix_transformer'
INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.local.image:'Docker Compose' found using Docker CLI.
INFO:sagemaker.local.local_session:Starting processing job
INFO:sagemaker.local.image:Using the long-lived AWS credentials found in session
INFO:sagemaker.local.image:docker compose file: 
networks:
  sagemaker-local:
    name: sagemaker-local
services:
  algo-1-zyuqd:
    container_name: 2

time="2024-07-24T20:16:26+01:00" level=warning msg="/private/var/folders/w1/813808f13m14dv7gd5ystxbc0000gn/T/tmpt8p5zqi_/docker-compose.yaml: `version` is obsolete"
time="2024-07-24T20:16:26+01:00" level=warning msg="a network with name sagemaker-local exists but was not created for project \"tmpt8p5zqi_\".\nSet `external: true` to use an existing network"
 Container 2403kcscc9-algo-1-zyuqd  Creating
 Container 2403kcscc9-algo-1-zyuqd  Created
Attaching to 2403kcscc9-algo-1-zyuqd
2403kcscc9-algo-1-zyuqd  | INFO:root:['output', 'input']
2403kcscc9-algo-1-zyuqd  | INFO:root:['data', 'code']


INFO:sagemaker.local.image:===== Job Complete =====


2403kcscc9-algo-1-zyuqd exited with code 0
Aborting on container exit...
 Container 2403kcscc9-algo-1-zyuqd  Stopping
 Container 2403kcscc9-algo-1-zyuqd  Stopped


INFO:sagemaker.local.entities:Pipeline step 'user_item_matrix_transformer' SUCCEEDED.
INFO:sagemaker.local.entities:Starting pipeline step: 'item_similarity_matrix_transformer'
INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.local.image:'Docker Compose' found using Docker CLI.
INFO:sagemaker.local.local_session:Starting processing job
INFO:sagemaker.local.image:Using the long-lived AWS credentials found in session
INFO:sagemaker.local.image:docker compose file: 
networks:
  sagemaker-local:
    name: sagemaker-local
services:
  algo-1-n17pa:
    container_name: p8n4w63qi7-algo-1-n17pa
    entrypoin

...

INFO:sagemaker.local.entities:Pipeline step 'item_similarity_matrix_transformer' SUCCEEDED.
INFO:sagemaker.local.entities:Starting pipeline step: 'user_similarity_matrix_transformer'
INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.local.image:'Docker Compose' found using Docker CLI.
INFO:sagemaker.local.local_session:Starting processing job
INFO:sagemaker.local.image:Using the long-lived AWS credentials found in session
INFO:sagemaker.local.image:docker compose file: 
networks:
  sagemaker-local:
    name: sagemaker-local
services:
  algo-1-t1gzp:
    container_name: jw67adc1ir-algo-1-t1gzp
    ent

time="2024-07-24T20:16:33+01:00" level=warning msg="/private/var/folders/w1/813808f13m14dv7gd5ystxbc0000gn/T/tmp0uopatez/docker-compose.yaml: `version` is obsolete"
time="2024-07-24T20:16:33+01:00" level=warning msg="a network with name sagemaker-local exists but was not created for project \"tmp0uopatez\".\nSet `external: true` to use an existing network"
 Container jw67adc1ir-algo-1-t1gzp  Creating
 Container jw67adc1ir-algo-1-t1gzp  Created
Attaching to jw67adc1ir-algo-1-t1gzp


INFO:sagemaker.local.image:===== Job Complete =====


jw67adc1ir-algo-1-t1gzp exited with code 0
Aborting on container exit...
 Container jw67adc1ir-algo-1-t1gzp  Stopping
 Container jw67adc1ir-algo-1-t1gzp  Stopped


INFO:sagemaker.local.entities:Pipeline step 'user_similarity_matrix_transformer' SUCCEEDED.
INFO:sagemaker.local.entities:Pipeline execution 55497159-63ff-4182-a491-c3a80d9387db SUCCEEDED


In [709]:
steps = execution.list_steps()
steps

{'PipelineExecutionSteps': [{'EndTime': 1721848588.968956,
   'Metadata': {'ProcessingJob': {'Arn': 'user_item_matrix_transformer-1721848585-2b7b'}},
   'StartTime': 1721848585.223544,
   'StepName': 'user_item_matrix_transformer',
   'StepStatus': 'Succeeded'},
  {'EndTime': 1721848596.133906,
   'Metadata': {'ProcessingJob': {'Arn': 'user_similarity_matrix_transformer-1721848592-3dc1'}},
   'StartTime': 1721848592.548224,
   'StepName': 'user_similarity_matrix_transformer',
   'StepStatus': 'Succeeded'},
  {'EndTime': 1721848592.546102,
   'Metadata': {'ProcessingJob': {'Arn': 'item_similarity_matrix_transformer-1721848588-3b85'}},
   'StartTime': 1721848588.971316,
   'StepName': 'item_similarity_matrix_transformer',
   'StepStatus': 'Succeeded'}]}